## Load necessary libraries

In [10]:
pip install ipynb

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Download necessary libraries
import pandas as pd 
import numpy as np 
from datetime import date 

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from ipynb.fs.full.neuralNetwork import *


import copy
import matplotlib.pyplot as plt
import h5py
import scipy

from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.impute import SimpleImputer

# Step 1: Load dataset

In [15]:
regular_season = pd.read_csv('RegularSeasonStats.csv')
playoff = pd.read_csv('PlayoffStats.csv')

## Divide the data into training and test set

In [25]:
regular_season_copy = regular_season.copy()

# Define features and targets
X = regular_season_copy.drop(columns=['win'])
Y = regular_season_copy['win']

# Train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 28, stratify = Y)

X_train, X_test = X_train.T, X_test.T

# Step 2: Two-layer Neural Network

In [27]:
### Define the cosntant used through out
n_x = X.shape[0]
n_h = 4
n_y = 1
layer_dims = (n_x, n_h, n_y)


In [ ]:
def two_layer_model(X, Y, layer_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost = False):
    """
    Implements a two-layer neural network: LINEAR->RELU->LINEAR->SIGMOID

    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if home team wins, 0 if home team losses)
    """

(36, 24616)

In [29]:
Y_train.head()

30317    0
13233    1
28172    0
19382    0
27753    1
Name: win, dtype: int64